## Model Encoding and Replacing Null Values

### Importing Necessary Libraries

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import category_encoders as ce
from sklearn.linear_model import LinearRegression,Ridge
from sklearn.model_selection import train_test_split,RepeatedKFold,GridSearchCV
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


### Reading the first dataset of Monthly Food Retail Prices

In [6]:
df = pd.read_csv("Monthly_Food_Retail_Prices.csv")

### Checking the Dataset for NULL values

In [7]:
df.isnull().sum()

State                0
Centre               0
Commodity         9720
Variety         205578
Unit                 0
Category             0
Date                 0
Retail Price    568921
dtype: int64

### Replacing the Null Values in Commodity and Variety

In [8]:
df["Commodity"].fillna("Not Available",inplace=True)
df["Variety"].fillna("FAQ",inplace = True)
df.isnull().sum()

State                0
Centre               0
Commodity            0
Variety              0
Unit                 0
Category             0
Date                 0
Retail Price    568921
dtype: int64

## One Hot Encoding

In [9]:
def one_hot_encoding(df,col):
    one_hot_encoder=ce.OneHotEncoder(cols=col,return_df=True,use_cat_names=True)
    df_final = one_hot_encoder.fit_transform(df)
    return df_final

In [10]:
df = one_hot_encoding(df,"State")
df = one_hot_encoding(df,"Centre")
df = one_hot_encoding(df,"Commodity")
df = one_hot_encoding(df,"Variety")
df = one_hot_encoding(df,"Unit")
df = one_hot_encoding(df, 'Category')

In [11]:
df

,State_Andhra Pradesh,State_Arunachal Pradesh,State_Assam,State_Bihar,State_Goa,State_Gujarat,State_Haryana,State_Himachal Pradesh,State_Jammu Kashmir,State_Jharkhand,...,Unit_Dozen,Unit_80gm. Pkt,Unit_Litre,Unit_Kg,Unit_400/800 Gm,Unit_Peice,Unit_500gm. Pkt,Category_Food,Date,Retail Price
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,JAN-2001,24.0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,JAN-2001,26.0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,JAN-2001,25.5
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,JAN-2001,NaN
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,JAN-2001,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1147441,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,MAR-2021,NaN
1147442,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,MAR-2021,NaN
1147443,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,MAR-2021,NaN
1147444,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,MAR-2021,20.0


### Modifying the Date Column in The Dataset to construct separate Month and Year columns

In [12]:
df['Date'] = pd.to_datetime(df.Date)
df['Month'] = df["Date"].dt.month
df['Year'] = df["Date"].dt.year

### Removing Columns from Dataset that are redundant or not required

In [13]:
columns_to_be_removed = ["Category_Food", "Date"]

In [14]:
df.drop(columns_to_be_removed,axis = 1,inplace= True)

### Splitting the Dataset Into Test and Train
The test dataset contains all records that have Retail Prices as NaN or NULL. <br>
The train dataset contains all records that do not have null values

In [15]:
df_test = df[df["Retail Price"].isnull()==True]

In [16]:
df_train = df.dropna()
df_train

,State_Andhra Pradesh,State_Arunachal Pradesh,State_Assam,State_Bihar,State_Goa,State_Gujarat,State_Haryana,State_Himachal Pradesh,State_Jammu Kashmir,State_Jharkhand,...,Unit_Dozen,Unit_80gm. Pkt,Unit_Litre,Unit_Kg,Unit_400/800 Gm,Unit_Peice,Unit_500gm. Pkt,Retail Price,Month,Year
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,24.0,1,2001
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,26.0,1,2001
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,25.5,1,2001
10,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,28.0,1,2001
12,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,27.0,1,2001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1147436,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,21.0,3,2021
1147437,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,22.0,3,2021
1147438,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,25.0,3,2021
1147439,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,27.0,3,2021


### Predicting Retail Values for records that have their values as Nan (or in other words calculating Retail Prices for the Test Dataset

In [17]:
X_train = df_train.drop("Retail Price",axis = 1)
X_test = df_test.drop("Retail Price",axis = 1)
Y_train = df_train["Retail Price"]

### Using Linear Regression to Predict these values

In [18]:
lr = LinearRegression()

In [19]:
lr.fit(X_train,Y_train)

LinearRegression()

In [20]:
Y_pred = lr.predict(X_test)

### Equating the predicted values obtained as Retail Prices

In [21]:
df_test["Retail Price"] = Y_pred

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [22]:
df_test

,State_Andhra Pradesh,State_Arunachal Pradesh,State_Assam,State_Bihar,State_Goa,State_Gujarat,State_Haryana,State_Himachal Pradesh,State_Jammu Kashmir,State_Jharkhand,...,Unit_Dozen,Unit_80gm. Pkt,Unit_Litre,Unit_Kg,Unit_400/800 Gm,Unit_Peice,Unit_500gm. Pkt,Retail Price,Month,Year
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,3.889038,1,2001
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,-0.762695,1,2001
5,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,15.674805,1,2001
6,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,11.640625,1,2001
7,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,11.862549,1,2001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1147440,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,64.267334,3,2021
1147441,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,65.650879,3,2021
1147442,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,75.484131,3,2021
1147443,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,76.867920,3,2021


### Creating a Final DataFrame that contains both test and train dataset.
This final Dataframe will be used for final training of the model

In [23]:
df_final = pd.concat([df_train,df_test],axis = 0)

In [ ]:
df_final

,State_Andhra Pradesh,State_Arunachal Pradesh,State_Assam,State_Bihar,State_Goa,State_Gujarat,State_Haryana,State_Himachal Pradesh,State_Jammu Kashmir,State_Jharkhand,...,Unit_Dozen,Unit_80gm. Pkt,Unit_Litre,Unit_Kg,Unit_400/800 Gm,Unit_Peice,Unit_500gm. Pkt,Retail Price,Month,Year
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,24.000000,1,2001
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,26.000000,1,2001
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,25.500000,1,2001
10,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,28.000000,1,2001
12,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,27.000000,1,2001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1147440,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,64.267334,3,2021
1147441,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,65.650879,3,2021
1147442,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,75.484131,3,2021
1147443,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,76.867920,3,2021


#### Saving the Final Dataframe obtained as a csv file for further use.

In [24]:
df_final.to_csv('FinalDf.csv')